## Fancy Softmax Classifier
- cross_entropy
- one_hot
- reshape

In [2]:
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

(101, 16) (101, 1)


In [4]:
nb_classes = 7  # 0 ~ 6 클래스의 개수

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot:", Y_one_hot)  # rank를 더해서 차원이 하나 더 늘어나게 됨
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape one_hot:", Y_one_hot) # reshape을 통해 2치원으로 구함

one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)


In [5]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

In [6]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)

logits = tf.matmul(X, W) + b            # score에 해당
hypothesis = tf.nn.softmax(logits)      # logits을 softmax에 통과시키면 최종적으로 알고자 하는 가설이 나옴

06-1에서 사용한 방법 

    # Cross entropy cost/loss
    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
                                        Y는 one-hot으로 주어짐


In [7]:
# Cross entropy cost/loss 

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                 labels=tf.stop_gradient([Y_one_hot])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if step % 100 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:     0	Cost: 6.538	Acc: 28.71%
Step:   100	Cost: 0.815	Acc: 73.27%
Step:   200	Cost: 0.512	Acc: 86.14%
Step:   300	Cost: 0.373	Acc: 87.13%
Step:   400	Cost: 0.290	Acc: 90.10%
Step:   500	Cost: 0.236	Acc: 94.06%
Step:   600	Cost: 0.197	Acc: 94.06%
Step:   700	Cost: 0.169	Acc: 96.04%
Step:   800	Cost: 0.146	Acc: 97.03%
Step:   900	Cost: 0.129	Acc: 97.03%
Step:  1000	Cost: 0.115	Acc: 99.01%
Step:  1100	Cost: 0.104	Acc: 100.00%
Step:  1200	Cost: 0.094	Acc: 100.00%
Step:  1300	Cost: 0.087	Acc: 100.00%
Step:  1400	Cost: 0.080	Acc: 100.00%
Step:  1500	Cost: 0.074	Acc: 100.00%
Step:  1600	Cost: 0.069	Acc: 100.00%
Step:  1700	Cost: 0.065	Acc: 100.00%
Step:  1800	Cost: 0.061	Acc: 100.00%
Step:  1900	Cost: 0.058	Acc: 100.00%
Step:  2000	Cost: 0.055	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction